In [4]:
from typing import List
import pandas as pd
MFD2 = 'mfd2.0.dic'
nummap = dict()
mfd2 = dict()
wordmode = True
with open(MFD2, 'r') as f:
    for line in f.readlines():
        ent = line.strip().split()
        if line[0] == '%':
            wordmode = not wordmode
        elif len(ent) > 0:
            if wordmode:
                #print(ent)
                moral = nummap[ent[-1]]
                if (moral not in mfd2.keys()):
                    mfd2[moral] = []
                mfd2[moral].append(''.join([e for e in ent if e not in nummap.keys()]))

            # mfd2[]].append(ent[0])
            # elif ent[0] in nummap.keys():
            #     wordkey = ''.join([e for e in ent if e not in nummap.keys()])
            #     mfd2[wordkey] = [nummap[e] for e in ent if e in nummap.keys()]
            else:
                nummap[ent[0]] = ent[1]

# mfd2 = pd.DataFrame.from_dict(mfd2).T
# mfd2['foundation'] = mfd2[0]
# del mfd2[0]
# mfd2 = mfd2.T.to_dict()

In [5]:
mfd2.keys()

dict_keys(['care.virtue', 'care.vice', 'fairness.virtue', 'fairness.vice', 'loyalty.virtue', 'loyalty.vice', 'authority.virtue', 'authority.vice', 'sanctity.virtue', 'sanctity.vice'])

In [6]:
foundations = ['care','fairness','loyalty','authority','sanctity']

In [7]:
%pip install sentence-transformers

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.9.0-cp314-cp314-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached scikit_learn-1.7.2-cp314-cp314-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp314-cp314-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pillow-12.0.0-cp314-cp314-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached pyyaml-6.0.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/anagha/Documents/Fall 2025/anlp-project/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
word_embeddings = {}
for foundation, words in mfd2.items():
  word_embeddings[foundation] = model.encode(words)

In [10]:
import numpy as np

master_moral_vectors = {}
for foundation, embeddings in word_embeddings.items():
    master_moral_vectors[foundation] = np.mean(embeddings, axis=0)

print("Master Moral Vectors:")
for foundation, vector in master_moral_vectors.items():
    print(f"{foundation}: {vector[:5]}...") # Print first 5 elements for brevity

Master Moral Vectors:
care.virtue: [-0.02875727  0.03551083  0.00732217  0.02872039  0.00990646]...
care.vice: [-0.01678748  0.05873546  0.0029738   0.04724036  0.00010954]...
fairness.virtue: [-0.03395445  0.03819251 -0.01743832 -0.00404897 -0.01426102]...
fairness.vice: [-0.0485366   0.03494301 -0.00214647  0.01777522 -0.01023221]...
loyalty.virtue: [-0.02538481  0.00370543 -0.02385724  0.01727305 -0.01185597]...
loyalty.vice: [-0.04009189  0.01627939 -0.02847462  0.02501947 -0.01733256]...
authority.virtue: [-0.02727246  0.03400709 -0.01090748  0.01084201 -0.02491886]...
authority.vice: [-0.02178937  0.02740643 -0.01197635  0.01206735 -0.01943187]...
sanctity.virtue: [-0.01853777  0.04013454 -0.01779349  0.02171165 -0.02596272]...
sanctity.vice: [-0.02264338  0.02221924 -0.00735809  0.01935911 -0.0086274 ]...


In [11]:
all_embeddings = []
embedding_labels = []

for foundation, embeddings in word_embeddings.items():
    all_embeddings.extend(embeddings)
    embedding_labels.extend([f'{foundation} - word'] * len(embeddings))

for foundation, vector in master_moral_vectors.items():
    all_embeddings.append(vector)
    embedding_labels.append(f'{foundation} - master')

print(f"Total number of embeddings: {len(all_embeddings)}")
print(f"Total number of labels: {len(embedding_labels)}")

Total number of embeddings: 2114
Total number of labels: 2114


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def translate_dict_to_kannada(input_dict: dict) -> dict:
    """
    Translates the string values in a dictionary of lists to Kannada.

    Args:
        input_dict: A dictionary where keys are strings and values are lists of strings.
                    Example: {'moral': ['word1', 'word2'], 'moral2': ['word3']}

    Returns:
        A new dictionary with the same keys but with translated string values.
    """
    # Define the model name and target language
    model_name = "sarvamai/sarvam-translate"
    tgt_lang = "Kannada"

    # Load the tokenizer and model
    # The .to('cuda:0') part moves the model to the GPU for faster inference
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda:0')

    translated_dict = {}
    # Iterate over each key-value pair in the input dictionary
    for key, words in input_dict.items():
        translated_words = []
        # Iterate over each word in the list
        for word in words:
            # Create the prompt for the model using a chat template
            messages = [
                {"role": "system", "content": f"Translate the text below to {tgt_lang}."},
                {"role": "user", "content": word}
            ]

            # Apply the chat template to structure the conversation
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            # Tokenize the input and move it to the model's device (GPU)
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

            # Generate the translation
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=128,  # Increased token limit for potentially longer words
                do_sample=True,
                temperature=0.01,
                num_return_sequences=1
            )

            # Decode the generated output to get the translated text
            output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
            output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
            translated_words.append(output_text.strip())

        # Assign the list of translated words to the corresponding key
        translated_dict[key] = translated_words

    return translated_dict

# --- Example Usage ---


# 2. Call the function to get the translated dictionary
#    (This requires a machine with a compatible GPU and transformers installed)
translated_dictionary = translate_dict_to_kannada(mfd2["care.virtue"])

# 3. Print the result
print(translated_dictionary)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]